In [29]:
from skimage.io import imread, imsave
from skimage.measure import regionprops, regionprops_table
from skimage.measure import shannon_entropy as shentr
import numpy as np
import pandas as pd

In [56]:
labels = imread("test_images/test_tracked.tif")
flous = imread("test_images/test_cdc.tif")
phases = imread("test_images/test_cdc.tif")
intensity_ims = [flous, phases]
for imtype in [labels] + intensity_ims:
    print(imtype.shape, imtype.dtype)

(10, 520, 692) uint16
(10, 520, 692) uint16
(10, 520, 692) uint16


In [57]:
def shannon_entropy(label, intensity_im):
    return shentr(intensity_im, base = 2)

def image_variance(label, intensity_im):
    return np.var(intensity_im)

def image_median(label, intensity_im):
    return np.median(intensity_im)


In [58]:
def getBirthFrame(trackedMasks, cellVal):
    return np.min((np.nonzero(trackedMasks == cellVal))[0])

def getDeathFrame(trackedMasks, cellVal):
    return np.max((np.nonzero(trackedMasks == cellVal))[0])

In [65]:
PROPS = ["area", "axis_major_length", "axis_minor_length", "centroid", "eccentricity", "intensity_max", "intensity_mean", "intensity_min", "label"]
extra_properties = [shannon_entropy, image_variance, image_median]

In [70]:
total_labels = np.unique(labels)
total_labels = total_labels[total_labels!=0]
total_data = {"labels": total_labels, "birth": [], "death": []}
for val in total_labels:
    total_data["birth"].append(getBirthFrame(labels, val))
    total_data["death"].append(getDeathFrame(labels, val))


for i in range(labels.shape[0]):
    label = labels[i,:,:]
    intensity_im = np.stack([im[i] for im in intensity_ims], -1)
    im_data = regionprops_table(label, intensity_im, properties = PROPS, extra_properties = extra_properties)

    # add zeros for cells not present in im data
    for label in total_labels:
        if label not in im_data["label"]:
            for property in im_data:
                im_data[property] = list(im_data[property])
                if property != "label":
                    # prevent index out of bound error
                    if label-1 < len(im_data[property]):
                        im_data[property][label-1] = 0
                    else:
                        im_data[property].append(0)
    
    for property in im_data:
        if property != "label":
            if property not in total_data:
                total_data[property] = [[prop] for prop in im_data[property]]
            else:
                for label in total_labels:
                    total_data[property][label-1].append(im_data[property][label-1])



In [71]:
total_data

{'labels': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140], dtype=uint16),
 'birth': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [73]:
df = pd.DataFrame.from_dict(total_data)
df

,labels,birth,death,area,axis_major_length,axis_minor_length,centroid-0,centroid-1,eccentricity,intensity_max-0,...,intensity_mean-0,intensity_mean-1,intensity_min-0,intensity_min-1,shannon_entropy-0,shannon_entropy-1,image_variance-0,image_variance-1,image_median-0,image_median-1
0,1,0,9,"[401, 393, 401, 398, 397, 398, 399, 412, 408, ...","[23.84153499983216, 23.436457730157393, 23.746...","[21.46738546681212, 21.436100399431016, 21.584...","[65.38653366583542, 65.29516539440203, 65.1122...","[135.59102244389027, 135.49872773536896, 135.4...","[0.4350224586025691, 0.4042520528430813, 0.416...","[315.0, 263.0, 275.0, 283.0, 295.0, 279.0, 279...",...,"[192.34663341645884, 190.9592875318066, 189.56...","[192.34663341645884, 190.9592875318066, 189.56...","[155.0, 147.0, 143.0, 155.0, 155.0, 159.0, 155...","[155.0, 147.0, 143.0, 155.0, 155.0, 159.0, 155...","[3.762857663662809, 3.726313108148297, 3.70130...","[3.762857663662809, 3.726313108148297, 3.70130...","[7024.896776716024, 6516.488216500805, 7329.25...","[7024.896776716024, 6516.488216500805, 7329.25...","[183.0, 183.0, 183.0, 183.0, 187.0, 183.0, 183...","[183.0, 183.0, 183.0, 183.0, 187.0, 183.0, 183..."
1,2,0,9,"[47, 79, 104, 136, 159, 184, 211, 230, 261, 282]","[8.074649332606466, 10.372844556473108, 11.848...","[7.431269023270903, 9.716622266342762, 11.1861...","[58.59574468085106, 57.53164556962025, 57.0769...","[149.2340425531915, 149.50632911392404, 150.26...","[0.39116407181756324, 0.3500352845135413, 0.32...","[283.0, 303.0, 319.0, 303.0, 303.0, 295.0, 303...",...,"[199.68085106382978, 199.65822784810126, 195.8...","[199.68085106382978, 199.65822784810126, 195.8...","[163.0, 159.0, 155.0, 151.0, 155.0, 155.0, 155...","[163.0, 159.0, 155.0, 151.0, 155.0, 155.0, 155...","[3.8419365814945867, 3.8199471914335588, 4.030...","[3.8419365814945867, 3.8199471914335588, 4.030...","[8277.011474609375, 7404.837099999999, 6974.34...","[8277.011474609375, 7404.837099999999, 6974.34...","[185.0, 185.0, 187.0, 183.0, 187.0, 179.0, 179...","[185.0, 185.0, 187.0, 183.0, 187.0, 179.0, 179..."
2,3,0,9,"[274, 279, 282, 293, 283, 290, 299, 301, 303, ...","[21.89721881076201, 21.699532079263516, 22.070...","[16.004278643289474, 16.47620321570744, 16.361...","[68.78467153284672, 68.7921146953405, 68.73758...","[172.05109489051094, 172.4336917562724, 172.42...","[0.6825040645847436, 0.6507542624177036, 0.671...","[215.0, 227.0, 227.0, 227.0, 219.0, 235.0, 231...",...,"[190.27007299270073, 190.05376344086022, 190.2...","[190.27007299270073, 190.05376344086022, 190.2...","[159.0, 159.0, 159.0, 159.0, 151.0, 163.0, 159...","[159.0, 159.0, 159.0, 159.0, 151.0, 163.0, 159...","[3.5690252095568087, 3.590255258633952, 3.5874...","[3.5690252095568087, 3.590255258633952, 3.5874...","[5559.253932823129, 6286.3723842478175, 6122.3...","[5559.253932823129, 6286.3723842478175, 6122.3...","[187.0, 187.0, 187.0, 183.0, 187.0, 189.0, 187...","[187.0, 187.0, 187.0, 183.0, 187.0, 189.0, 187..."
3,4,0,9,"[364, 364, 359, 348, 342, 347, 359, 359, 361, ...","[23.358501085097966, 23.32859827733263, 23.100...","[19.95108835109769, 19.986132786088827, 19.861...","[71.30494505494505, 71.3076923076923, 71.90529...","[154.7445054945055, 154.62912087912088, 154.50...","[0.5200671635269756, 0.5157777848716929, 0.510...","[271.0, 263.0, 283.0, 263.0, 275.0, 255.0, 259...",...,"[194.32967032967034, 193.83516483516485, 192.9...","[194.32967032967034, 193.83516483516485, 192.9...","[159.0, 159.0, 151.0, 163.0, 159.0, 163.0, 159...","[159.0, 159.0, 151.0, 163.0, 159.0, 163.0, 159...","[3.726876360785678, 3.751784345620417, 3.70998...","[3.726876360785678, 3.751784345620417, 3.70998...","[7227.655500259335, 7157.5755907908215, 6637.3...","[7227.655500259335, 7157.5755907908215, 6637.3...","[187.0, 187.0, 187.0, 187.0, 187.0, 187.0, 185...","[187.0, 187.0, 187.0, 187.0, 187.0, 187.0, 185..."
4,5,0,9,"[295, 306, 294, 302, 309, 320, 330, 343, 347, ...","[22.612331514822355, 22.890621678927694, 22.59...","[16.612333094277176, 1

In [50]:
len(im_data["label"])

140

In [43]:
df[np.logical_and(df["label"] == 1, df["time"]==0)]

,label,intensity_mean-0,intensity_mean-1,shannon_entropy-0,shannon_entropy-1,image_variance-0,image_variance-1,image_median-0,image_median-1,time
0,1,192.346633,192.346633,3.762858,3.762858,7024.896777,7024.896777,183.0,183.0,0
